# Presentation: Analysis of PolyRatings data

In [1]:
ls /data301/share/robert_h/

polyratings_profiles/  polyratings_reviews.csv


## I. Collection

## II. Analysis

## III. Machine Learning  

## IV. Conclusions